# 1. Import Required Libraries
This section imports all necessary libraries for data analysis, preprocessing, modeling, and explainability.

In [10]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
import shap
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

# 2. Load and Inspect Data
Load the synthetic farmer data from `/data/farmers_data.csv` and inspect its structure.

In [11]:
# Load the data
data_path = '../data/farmers_data.csv'
df = pd.read_csv(data_path)
print('Shape:', df.shape)
df.head()

Shape: (100, 27)


,farmer_id,37203063,phone_number,region,age,education_level,gender,marital_status,dependents_count,household_income_sources,...,past_crop_performance_index,agroclimatic_zone,rainfall_forecast_next_90d,historical_drought_risk,cooperative_member,mobile_advisory_usage,loan_default_past,input_use_history,has_insurance_before,willing_to_subscribe_insurance
0,F0001,85227106.0,+1-437-579-8111x468,Kiambu,19,Secondary,Male,Single,1,"['Livestock', 'Small Business']",...,0.32,Kiambu_Z1,300.6,0.70,False,True,False,NaN,False,True
1,F0002,4326060.0,+1-691-272-7095,Embu,62,Primary,Female,Single,1,"['Small Business', 'Livestock']",...,0.68,Embu_Z2,77.6,0.29,False,True,True,Regular,True,False
2,F0003,12555605.0,+1-590-768-2353x7614,Kiambu,32,Secondary,Female,Married,3,"['Small Business', 'Livestock']",...,0.67,Kiambu_Z1,107.2,0.38,True,False,True,Regular,True,False
3,F0004,91497770.0,(170)850-1178x888,Kitale,35,NaN,Female,Single,5,"['None', 'Remittance']",...,0.60,Kitale_Z3,189.8,0.22,True,False,True,Regular,True,True
4,F0005,37269296.0,534-187-4647,Embu,61,Primary,Male,Married,3,"['None', 'Small Business']",...,0.83,Embu_Z3,169.1,0.29,True,False,True,NaN,False,True


# 3. Preprocessing
Encode categorical variables, normalize numeric columns, and split the data into train and test sets.

In [13]:
# Identify target and features
target_col = 'loan_class' if 'loan_class' in df.columns else df.columns[-1]
X = df.drop(columns=[target_col])
y = df[target_col]

# Encode categorical features
def encode_features(X):
    X_encoded = X.copy()
    for col in X_encoded.select_dtypes(include=['object']).columns:
        if X_encoded[col].nunique() < 10:
            X_encoded[col] = LabelEncoder().fit_transform(X_encoded[col].astype(str))
        else:
            X_encoded = pd.get_dummies(X_encoded, columns=[col], drop_first=True)
    return X_encoded

X_encoded = encode_features(X)

# Impute or drop missing values (fix for NaN)
X_encoded = X_encoded.dropna()
if len(X_encoded) != len(y):
    y = y.loc[X_encoded.index]

# Clean column names for XGBoost compatibility
X_encoded.columns = [str(c).replace('[','').replace(']','').replace('<','').replace('>','').replace("'","").replace(' ', '_') for c in X_encoded.columns]

# Normalize numeric columns
scaler = StandardScaler()
numeric_cols = X_encoded.select_dtypes(include=[np.number]).columns
X_encoded[numeric_cols] = scaler.fit_transform(X_encoded[numeric_cols])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

Train shape: (79, 245) Test shape: (20, 245)


# 4. Model Training
Train Logistic Regression, Random Forest, and XGBoost classifiers. Evaluate each with accuracy, F1-score, precision, recall, and confusion matrix.

In [14]:
# Train and evaluate models
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    return {'accuracy': acc, 'f1': f1, 'precision': prec, 'recall': rec, 'confusion_matrix': cm}

results = {}

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
results['LogisticRegression'] = evaluate_model(lr, X_test, y_test)

# Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
results['RandomForest'] = evaluate_model(rf, X_test, y_test)

# XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train, y_train)
results['XGBoost'] = evaluate_model(xgb, X_test, y_test)

# Print metrics
def print_metrics(results):
    print('Model\tAccuracy\tF1\tPrecision\tRecall')
    for name, res in results.items():
        print(f"{name}\t{res['accuracy']:.3f}\t{res['f1']:.3f}\t{res['precision']:.3f}\t{res['recall']:.3f}")
print_metrics(results)

Model	Accuracy	F1	Precision	Recall
LogisticRegression	0.550	0.533	0.592	0.550
RandomForest	0.650	0.647	0.648	0.650
XGBoost	0.800	0.800	0.816	0.800


# 5. Model Selection and Saving
Compare metrics, select the best model (prefer RandomForest), and save it to `/models/credit_model.pkl`.

In [15]:
# Select best model (prefer RandomForest if close)
best_model_name = max(results, key=lambda k: results[k]['f1'])
if best_model_name != 'RandomForest' and results['RandomForest']['f1'] >= results[best_model_name]['f1'] - 0.01:
    best_model_name = 'RandomForest'
best_model = {'LogisticRegression': lr, 'RandomForest': rf, 'XGBoost': xgb}[best_model_name]

# Save the best model
os.makedirs('../models', exist_ok=True)
joblib.dump(best_model, '../models/credit_model.pkl')
print(f"Best model: {best_model_name} saved to ../models/credit_model.pkl")

Best model: XGBoost saved to ../models/credit_model.pkl


# 6. Explainability
Show feature importance using SHAP (if XGBoost or RandomForest) or model coefficients. Print top 5 influential features.

In [18]:
# Feature importance or SHAP
feature_names = X_encoded.columns
if best_model_name in ['RandomForest', 'XGBoost']:
    try:
        explainer = shap.TreeExplainer(best_model)
        shap_values = explainer.shap_values(X_train)
        shap.summary_plot(shap_values, X_train, feature_names=feature_names, show=False)
        plt.title('SHAP Feature Importance')
        plt.show()
        # Get mean absolute SHAP values for ranking
        shap_importance = np.abs(shap_values).mean(axis=0)
        top_idx = np.argsort(shap_importance)[::-1][:5]
        print('Top 5 features by SHAP:')
        for i in top_idx:
            print(f"{feature_names[i]}: {shap_importance[i]:.4f}")
    except Exception as e:
        print('SHAP explainability failed:', e)
        if hasattr(best_model, 'feature_importances_'):
            importances = best_model.feature_importances_
            idx = np.argsort(importances)[::-1][:5]
            print('Top 5 features by model importance:')
            for i in idx:
                print(f"{feature_names[i]}: {importances[i]:.4f}")
else:
    if hasattr(best_model, 'coef_'):
        coefs = best_model.coef_[0]
        idx = np.argsort(np.abs(coefs))[::-1][:5]
        print('Top 5 features by coefficient:')
        for i in idx:
            print(f"{feature_names[i]}: {coefs[i]:.4f}")

SHAP explainability failed: could not convert string to float: '[4.556962E-1]'
Top 5 features by model importance:
region: 0.1142
agroclimatic_zone_Kiambu_Z1: 0.0910
input_use_history: 0.0859
mobile_advisory_usage: 0.0623
loan_default_past: 0.0607


# 7. Summary Output
Print the best model name, top features, and a short interpretation of the results.

In [8]:
print(f"Best Model: {best_model_name}")
print("Key features: See above for top 5 influential features.")
print("Interpretation: The selected model predicts loan eligibility class (A/B/C) for smallholder farmers using demographic, behavioral, and farm-related features. Top features indicate the most influential factors in credit scoring for this dataset.")

Best Model: XGBoost
Key features: See above for top 5 influential features.
Interpretation: The selected model predicts loan eligibility class (A/B/C) for smallholder farmers using demographic, behavioral, and farm-related features. Top features indicate the most influential factors in credit scoring for this dataset.


# 8. Model Test and Example Prediction
Test the model on the test set and show accuracy. Also, show a sample farmer's name (ID) and their predicted loan class.

In [20]:
# Test model accuracy on test set
from sklearn.metrics import accuracy_score

y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test set accuracy: {acc:.3f}")

# Show a sample farmer's ID and their predicted loan class
# Find the index in the original df that matches the first test sample
sample_idx = X_test.index[0]
farmer_id = df.loc[sample_idx, 'farmer_id'] if 'farmer_id' in df.columns else sample_idx
predicted_class = best_model.predict([X_test.iloc[0]])[0]
print(f"Sample Farmer ID: {farmer_id}, Predicted Loan Class: {predicted_class}")

Test set accuracy: 0.800
Sample Farmer ID: F0043, Predicted Loan Class: 1
